# Исследование надежности заемщиков

**Цель исследования** - сбор данных для построения модели оценки способности потенциального заёмщика вовремя вернуть кредит банку.

**Описание данных**  - датафрейм с информацией клиентов, где:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

<div align='right'>Проектную работу выполнил студент Гагиев Денис. </div>

---

## Обзор данных

In [1]:
# импорт библиотек
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# загрузка данных
try:
    data = pd.read_csv('ссылка скрыта')
except:
    data = pd.read_csv('ссылка скрыта')

Ознакомимся с данными:

In [3]:
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Посмотрим на количество пропущенных значений для каждого столбца:

In [4]:
# колличество пропуском
display(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведим медианное значение трудового стажа days_employed в днях.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

Выведим перечень уникальных значений столбца `children`:

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`:

In [10]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены:

In [11]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`

In [12]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв.

In [13]:
data['education'] = data['education'].str.lower()

Посмотрим колличество дубликатов:

In [14]:
data.duplicated().sum()

71

Удалим дубликаты:

In [15]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [16]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [17]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведим на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [18]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.

In [19]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [20]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследовательский анализ данных

Ответим на следующий ряд вопросов:

#### 1. Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [21]:
data_child_debt = data.pivot_table(index='children', columns='debt', values='education', aggfunc='count') #строим сводную таблицу
data_child_debt['доля должников (%)'] = (data_child_debt[1] / (data_child_debt[0] + data_child_debt[1]) * 100)
data_child_debt = data_child_debt.rename(columns={1:'клиентов с задолженностью',0:'клиентов без задолженности'})
display(data_child_debt)

debt,клиентов без задолженности,клиентов с задолженностью,доля должников (%)
children,,,
0,13028.0,1063.0,7.543822
1,4364.0,444.0,9.234609
2,1858.0,194.0,9.454191
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


**Вывод:** кредитополучатели без детей являются более ответственными, чем клиенты, у которых есть дети. Видимо, наличие ребенка накладывает на клиента дополнительные финансовое бремя (обязательства) и клиенты либо менее ответственно относятся к своим кредитным обязательствам, либо не имеют возможности в срок оплачивать обязательства. 

Ну и что ещё видно в наших данных (агрегация «по count» нам в этом помогает), то, что по некоторым группам заемщиков у нас недостаточно данных. Говорят, что выборка не сбалансирована, классы в выборке имеют разные размеры.

#### 2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [22]:
data_family_debt = data.pivot_table(index='family_status', columns='debt', values='education', aggfunc='count')
data_family_debt['доля должников (%)'] = (data_family_debt[1] / (data_family_debt[0] + data_family_debt[1]) * 100)
data_family_debt = data_family_debt.sort_values(0, ascending=False).rename(columns={1:'клиентов с задолженностью',0:'клиентов без задолженности'})
display(data_family_debt)

debt,клиентов без задолженности,клиентов с задолженностью,доля должников (%)
family_status,,,
женат / замужем,11334,927,7.560558
гражданский брак,3749,385,9.313014
Не женат / не замужем,2523,273,9.763948
в разводе,1105,84,7.064760
вдовец / вдова,888,63,6.624606


**Вывод:** кредитному отделу следует с осторожностью относиться к клиентам, которые не состоят или не состояли в браке. Согласно данным такие клиенты менее ответственные. Получается, что узаконенные отношения ведут к более ответственному отношению к своим обязательствам. Ну вроде как совпадает со здравым смыслом - семейные люди более скрупулёзно берегут свой семейное спокойствие и несут за него ответственность.

#### 3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [23]:
#переименование значений для лучшего понимания сводной таблицы:
data['total_income_category'] = data['total_income_category'].replace('C', "C (50001–200000 рублей)") \
.replace('B', "B (200001–1000000)").replace('D', "D (30001–50000)") \
.replace('A', "A (1000001 и выше)").replace('E', "E (0–30000)")

#создание сводной таблицы:
data_income_debt = data.pivot_table(index='total_income_category', \
    columns='debt', values='education', aggfunc='count')

#расчет доли должников:
data_income_debt['доля должников (%)'] = (data_income_debt[1] / \
    (data_income_debt[0] + data_income_debt[1]) * 100)

#сортировка и переименование столбцов:
data_income_debt = data_income_debt.sort_values(0, ascending=False) \
    .rename(columns={1:'клиентов с задолженностью',0:'клиентов без задолженности'})

data_income_debt

debt,клиентов без задолженности,клиентов с задолженностью,доля должников (%)
total_income_category,,,
C (50001–200000 рублей),14568,1353,8.498210
B (200001–1000000),4660,354,7.060231
D (30001–50000),328,21,6.017192
A (1000001 и выше),23,2,8.000000
E (0–30000),20,2,9.090909


**Вывод:** исходя из здравого смысла логично предположить, что чаще всего должниками становятся люди с небольшим доходом, которым финансовые проблемы не позволяют вовремя осуществлять платежи. Но глядя на наши результаты лично мой вывод таков: выборка по заемщикам, как с наименьшим доходом, так и с наибольшим крайне недостаточна для формирования однозначных выводов. Можно думать, что E-категория самая низкооплачиваемая не может корректно распоряжаться деньгами и «влезает» в кредиты, но тогда неясно почему у А-категории также достаточно высокий процент задолженности. В общем вывод по группам А и Е неочевиден, мало данных.

Если же взять для анализа только группы В, С и D, то по полученным данным не наблюдаем зависимости "те, кто получает больше ЗП, лучше возвращает кредит». В целом это логично, так как материальный достаток - не единственная переменная, которая влияет на качество кредитной истории.

#### 4. Как разные цели кредита влияют на его возврат в срок?

In [24]:
data_purpose_debt = data.pivot_table(index='purpose_category', columns='debt', values='education', aggfunc='count')
data_purpose_debt['доля должников (%)'] = (data_purpose_debt[1] / (data_purpose_debt[0] + data_purpose_debt[1]) * 100)
data_purpose_debt = data_purpose_debt.sort_values(0, ascending=False).rename(columns={1:'клиентов с задолженностью',0:'клиентов без задолженности'})
data_purpose_debt

debt,клиентов без задолженности,клиентов с задолженностью,доля должников (%)
purpose_category,,,
операции с недвижимостью,9971,780,7.255139
операции с автомобилем,3879,400,9.347978
получение образования,3619,369,9.252758
проведение свадьбы,2130,183,7.911803


**Вывод:** есть зависимость. Среди клиентов целью кридита которых является "операции с недвижимостью" или "проведение свадьбы" меньше всего должников. 

#### 5. Приведите возможные причины появления пропусков в исходных данных.

**Ответ:** с высокой долей вероятности можно утверждать, что произошла техническая ошибка при формировании таблицы с исходными данными (либо некий сбой во время выгрузки данных на анализ, либо при объединении данных в единый файл, который также должен был передаваться на анализ, либо эти данные не собирались).

#### 6. Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

**Ответ:** медианное значение является более подходящее решением для заполнения пропусков, если имеются индивидуальные данные с сильным отклонением (4, 3, 5, 84, 4, 8), где метод mean может исказить понимание данных. 

---

<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    
    
# **Заключение**
---
**Цель исследования** - сбор данных для построения модели оценки способности потенциального заёмщика вовремя вернуть кредит банку.

**В рамках исследования выдвинуты следующие гипотезы:**
- колличество детей у заемщика влияет на факт погашения кредита в срок;
- семейное положение заемщика влияет влияет на факт погашения кредита в срок.

**Данные**: для проверки выдвинутых гипотез задействована статистика о платёжеспособности клиентов банка. Для точности анализа произведена предобработках полученного массива данных:
- выявлены и заполнены пропуски в данных;
- обработаны аномальные значения;
- обработаны явные и неявные дубликаты значений;
- обработы неккоректные типы данных.
---
**Гипотеза 1 - колличество детей у заемщика влияет на факт погашения кредита в срок.**

Для проверки гипотезы [оценена взаимосвязь между наличием детей и фактом просрочки задолженности.](#children_test)

Выявлена четкая взаимосвязь между наличием детей и их полным отсутствием. Клиенты без детей являются более платежеспособными. В тоже время незначительной и нелогической является взаимосвязь касательно колличества детей, согласно проведенному анализу колличество детей на оказывает флияние на факт погашение кредита вовремя.

Таким образом - **гипотеза подтверждена частично** и влияние на факт оплаты кредита в срок оказывает только полное отсутствие детей или их наличие. 

**Гипотеза 2 - семейное положение заемщика влияет влияет на факт погашения кредита в срок.**

Здесь также [оценена взаимосвязь между семейным положением и фактом просрочки задолженности.](#family_test)

Самая большая группа - это заемщики находящиеся в официальном браке, в тоже время они являются наиболее ответственными платильщиками. Чем не могут выделиться заемщики не состоящие в браке. Скорее всего это связанно с тем, что люди семейного порядка являются более обязательными и ответственными. 

Таким образом - **гипотеза подтвержена** и семейной положение действительно сказывается на факт оплаты кредита в срок. Кредит с большой вероятностью должен быть одобрен, если потенциальный заемщик состоит в официальном браке. 
    
**Заключение**: выдвинутые гипотезы подтверждают своё существование, при одобрении кредита важно обращать внимание на наличие детей (не колличество), а также на семейной положении, желательно наличие статуса официального брака. Также рекомендуется дополнить выборку  групп, где четверо и более детей и групп с доходом более 1 млн. рублей и доходом до 30 тыс. рублей, поскольку на данный момент выборка неудовлетворительна по своему колличеству.  

</div>

